In [5]:
# Librerías de extracción de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np

from time import sleep
import random
import datetime

import sys
import os

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

sys.path.append(os.path.abspath('../src'))
import funciones_auxiliares as fa

In [6]:
municipios = ["aranjuez", "arroyomolinos", "brunete", "chinchon", "alcobendas"]
df_definitivo = pd.DataFrame()

for munici in municipios:
    driver = webdriver.Chrome()
    url_wunder = "https://www.wunderground.com/history"
    driver.get(url_wunder)
    driver.maximize_window()

    #Denegamos las cookies
    iframe_cookies = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", '//*[@id="sp_message_iframe_1165301"]')))
    driver.switch_to.frame(iframe_cookies)

    sleep(3)
    try:
        driver.find_element("css selector", "#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column").click()
    except:
        print("No encuentro el boton de las cookies")

    driver.switch_to.default_content()
    # Introduce el municipio en la página principal
    sleep(3)
    driver.find_element("css selector", "#historySearch").send_keys(f"{munici}, Spain", Keys.ENTER)

    # Damos click en view
    sleep(3)
    driver.find_element("css selector", "#dateSubmit").click()
    driver.find_element("css selector", "#dateSubmit").click()

    # Pinchar en monthly de la página del municipio
    sleep(3)
    driver.find_element("css selector", "#inner-content > div.region-content-main > div.row > div:nth-child(1) > div:nth-child(1) > div > lib-link-selector > div > div > div > a:nth-child(3)").click()

    # Obtenemos los datos mensuales de cada region
    df_final=pd.DataFrame()

    for i in range(1,11):
        # Seleccionamos la caja de mes
        driver.find_element("css selector", "#monthSelection").click()
        sleep(3)
        # Seleccionamos el mes
        driver.find_element("css selector", f"#monthSelection > option:nth-child({i})").click()
        sleep(3)
        # Clicamos en View
        driver.find_element("css selector", "#dateSubmit").click()
        sleep(3)
        # Cogemos el html de la pagina actual
        html_table_page = driver.page_source

        sopa = BeautifulSoup(html_table_page, "html.parser")
        df=fa.df_datos_mes(sopa)
        df_final = pd.concat([df_final, df])
    
    df_final.insert(0,"municipio", munici)

    df_definitivo = pd.concat([df_definitivo, df_final])
    driver.quit()


In [7]:
df_definitivo

,municipio,mes,dia,max_temp,avg_temp,min_temp,max_rocio,avg_rocio,min_rocio,max_humedad,avg_humedad,min_humedad,max_viento,avg_viento,min_viento,max_presion,avg_presion,min_presion,lluvia
0,aranjuez,Jan,1,50,41.0,37,41,37.4,34,100,87.3,62,8,2.7,0,28.1,28.0,28.0,0.00
1,aranjuez,Jan,2,45,40.3,34,43,38.3,32,100,93.2,81,5,1.6,0,28.1,28.1,28.0,0.00
2,aranjuez,Jan,3,55,50.4,43,54,48.9,43,100,94.9,88,13,4.5,0,28.0,28.0,27.9,0.00
3,aranjuez,Jan,4,50,48.0,46,50,47.5,46,100,98.2,93,10,2.4,0,28.0,27.9,27.7,0.00
4,aranjuez,Jan,5,50,45.7,41,46,37.5,32,93,74.0,54,18,8.7,1,27.8,27.7,27.6,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,alcobendas,Oct,12,63,60.4,57,61,58.1,55,100,93.3,82,9,1.9,0,27.9,27.8,27.8,0.00
12,alcobendas,Oct,13,75,63.6,55,59,55.9,54,100,78.7,53,6,1.8,0,28.0,28.0,27.9,0.00
13,alcobendas,Oct,14,73,64.5,57,61,58.5,55,100,81.9,61,6,1.0,0,28.0,27.9,27.8,0.00
14,alcobendas,Oct,15,70,64.4,61,61,59.1,55,100,83.0,64,20,7.9,0,27.8,27.7,27.7,0.00


In [9]:
df_definitivo.to_csv("../datos/df_temperaturas.csv")